**Imports**

In [ ]:
import zipfile
import os

from keras import utils
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import Adam

**Unzip the training data**

In [ ]:
def unzip_to_cwd(zip_filepath, target_folder='train'):
    current_dir = os.getcwd()
    target_folder_path = os.path.join(current_dir, target_folder)

    if not os.path.exists(target_folder_path):
        os.makedirs(target_folder_path)

    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(target_folder_path)

zip_filepath = 'images_zipped.zip'
unzip_to_cwd(zip_filepath)

**Create training and validation sets**

In [ ]:
path_to_data = './train'

In [ ]:
train_data=utils.image_dataset_from_directory(
    path_to_data,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="training",
    shuffle=True,
    color_mode="rgb",
    image_size=(256,256),
    batch_size=64,
    seed=40,
)

In [ ]:
val_data=utils.image_dataset_from_directory(
    path_to_data,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="validation",
    color_mode="rgb",
    image_size=(256,256),
    batch_size=64,
    seed=40,
)

**Verify training and validation sets with labels (classes)**

In [ ]:
for images,lables in train_data.take(1):
  print('Amount of train images: ', images.shape)
  print('Amount of train labels: ', lables.shape)

In [ ]:
for images,lables in val_data.take(1):
  print('Amount of validation images: ', images.shape)
  print('Amount of validation labels: ', lables.shape)

In [ ]:
classes=train_data.class_names
print('The following classes are in the dataset:')
print(classes)

**Define normalization function and normalize both sets**

In [ ]:
def normalize(image, label):
  return image/255.0, label

In [ ]:
train_data = train_data.map(normalize)
val_data= val_data.map(normalize)

**Create one Tensor of the training images and labels**

In [ ]:
train_x=[]
train_y=[]

for image,label in train_data:
   train_x.append(image)
   train_y.append(label)

train_x = tf.concat(train_x, axis=0)
train_y = tf.concat(train_y, axis=0)

**Create one Tensor of the validation images and labels**

In [ ]:
val_x=[]
val_y=[]

for image,label in train_data:
   val_x.append(image)
   val_y.append(label)

val_x = tf.concat(val_x, axis=0)
val_y = tf.concat(val_y, axis=0)

**Create binary matrix representation of the labels (One-Hot Encoding format)**

In [ ]:
num_classes = 10
train_y = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)
val_y = tf.keras.utils.to_categorical(val_y, num_classes=num_classes)

**Define and verify convolution neural network**

In [ ]:
model = Sequential()
model.add(Conv2D(60, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(120, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(40, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

**Select optimizer and loss function for the training proces**

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Get the batch size and amount of epochs from environment variables**

In [ ]:
batch_size_param = int(os.getenv('batch_size'))
nr_of_epochs_param = int(os.getenv('nr_of_epochs'))

**Train the model**

In [ ]:
model.fit(train_x, train_y, batch_size=batch_size_param, epochs=nr_of_epochs_param,
          validation_data=(val_x,val_y))

**Evaluate the model**

In [ ]:
loss, accuracy = model.evaluate(val_x,val_y)
print('Test accuracy:', accuracy)

**Save the model**

In [ ]:
model.save('fracture_model.h5')